In [10]:
import numpy as np
import torch
import torch.nn.functional as F
import random

In [11]:
# opening the file in read mode 
my_file = open("../data/names.txt", "r") 
# reading the file 
data = my_file.read()
# split the names into a list
name_list = data.split('\n')
name_list = ["."+x for x in name_list]

chartoix = {char: ix for ix, char in enumerate(sorted(set("".join(name_list))))}
ixtochar = {ix: char for ix, char in enumerate(sorted(set("".join(name_list))))}

block_size = 3
def build_x_y(name_list, block_size):
    # Create a dataset with, for each example, all possible X and Y combinations we can extract from it
    y_list = []
    x_list = []
    for name in name_list:
        for target_ix, target_letter in enumerate(name):
            previous_letters = name[max(-(block_size)+target_ix,0):target_ix] # This will extract the previous 3 letters
            num_pads = block_size - len(previous_letters)
            previous_letters = "."*num_pads + previous_letters
            #print(previous_letters + "-->" + target_letter)
            y_list.append(chartoix[target_letter])
            x_list.append([chartoix[input_letter] for input_letter in previous_letters])
            
    X = torch.tensor(x_list) # Tensor with 3 dimensional list, input letter index
    Y = torch.tensor(y_list) # Tensor with 1 dimensional output
    return X,Y

ix_80_pct = int(0.8*len(name_list))
ix_90_pct = int(0.9*len(name_list))

X_train, Y_train = build_x_y(name_list[:ix_80_pct],block_size=block_size)
X_val, Y_val= build_x_y(name_list[ix_80_pct:ix_90_pct],block_size=block_size)
X_test, Y_test = build_x_y(name_list[ix_90_pct:],block_size=block_size)

X_train.shape, Y_train.shape, X_val.shape, Y_val.shape, X_test.shape, Y_test.shape

(torch.Size([182778, 3]),
 torch.Size([182778]),
 torch.Size([22633, 3]),
 torch.Size([22633]),
 torch.Size([22735, 3]),
 torch.Size([22735]))

In [12]:
class Linear:

    def __init__(self, fan_in, fan_out, bias = True):
        self.weight = torch.randn((fan_in, fan_out), generator = g) / fan_in**0.5 #Kaimin initialisation
        self.bias = torch.randn(fan_out) if bias else None

    def __call__(self, x): #Forward pass
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out

    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])

class BatchNorm1d:

    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps=eps
        self.momentum=momentum
        self.training=True
        #parameters for scaling the batchnorm
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        #running mean and std
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)

    def __call__(self, x): #Forward pass can be during training or inference depending on self.training
           
        if self.training:
            batch_mean_i = x.mean(0,keepdims=True) 
            batch_var_i = x.var(0,keepdims=True)

            with torch.no_grad():
                self.running_mean = (1-momentum)*self.running_mean + momentum*batch_mean_i
                self.running_var = (1-momentum)*self.running_var + momentum*batch_var_i

        else:
            
            batch_mean_i = self.running_mean
            batch_var_i = self.running_var

        x_hat = x - batch_mean_i / torch.sqrt(batch_var_i + self.eps)
        self.out = self.gamma * x_hat + self.beta

        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]

class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
    def parameters(self):
        return []


In [13]:
n_embd = 10
n_hidden = 100
vocab_size = len(chartoix)
g = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size,n_embd), generator=g)
layers = [
    Linear(block_size*n_embd, n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), Tanh(),
    Linear(n_hidden, vocab_size),
]

# We scale down the last layer's weights before giving the final output
with torch.no_grad():
    layers[-1].weight *= 0.1 #for the last layer only
    for layer in layers[:-1]: 
        if isinstance(layer,Linear): #for all other layers
            layer.weight *= 5/3 # This is the gain of the Kaimin init. The / sqrt(n) is already part of the class

parameters = [[C]] + [layer.parameters() for layer in layers]
flattened_params = [item for sublist in parameters for item in sublist]

In [14]:
print(f"Total number of params: {sum([params.nelement() for params in flattened_params])}")

Total number of params: 46497


In [15]:
for p in flattened_params:
    p.requires_grad = True

In [18]:
steps = 10000
batch_size = 32
lossi = []
for i in range(steps):
    ###### Minibatch #######
    minibatch_ix = torch.randint(0,len(X_train),(batch_size,), generator = g)
    Xb, Yb = X_train[minibatch_ix], Y_train[minibatch_ix]
    
    ###### Forward pass ########
    embs = C[Xb]
    x = embs.view(embs.shape[0],-1)
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x, Yb)
    
    ###### Backward pass ########
    for layer in layers:
        layer.out.retain_grad()
    for p in flattened_params:
        p.grad = None
    loss.backward()

    lr = 0.1 if i < 50000 else 0.01
    for p in flattened_params:
        p.data += -lr*p.grad

    if i % 1000 == 0:
        print(f"Iteration {i}, Loss: {loss.item()}")
    lossi.append(loss.item())

Iteration 0, Loss: 1.8782804012298584
Iteration 1000, Loss: 2.385084390640259
Iteration 2000, Loss: 2.0015838146209717
Iteration 3000, Loss: 2.2290639877319336
Iteration 4000, Loss: 2.041823148727417
Iteration 5000, Loss: 1.4566975831985474
Iteration 6000, Loss: 1.710499882698059
Iteration 7000, Loss: 2.0581700801849365
Iteration 8000, Loss: 2.0209033489227295
Iteration 9000, Loss: 1.86842942237854
